# Image Analysis with Python - <font color='teal'>Tutorial Pipeline Section 1</font>

*originally created in 2016*<br>
*updated and converted to a Jupyter notebook in 2017*<br>
*updated and converted to python 3 in 2018*<br>
*by Jonas Hartmann (Gilmour group, EMBL Heidelberg)*<br>
*updated in 2022 by Cheng-Yu Huang*<br>

##  Table of Contents

1. [About this Tutorial](#about)
2. [Setup](#setup)
3. [Importing Modules & Packages](#import)
4. [Loading & Handling Image Data](#load)
5. [Preprocessing](#prepro)
6. [Manual Thresholding & Threshold Detection](#thresh)
7. [Adaptive Thresholding](#adaptive)
8. [Improving Masks with Binary Morphology](#morpho)
9. [Connected Components Labeling](#label)
10. [Cell Segmentation by Seeding & Expansion](#seg)

##  About this Tutorial <a id=about></a>

*This tutorial aims to teach the basics of (bio-)image processing with python, in particular the analysis of fluorescence microscopy data. The tutorial is self-explanatory and follows a "learning by doing" philosophy. It consists of step by step instructions that guide students through the construction of a 2D single-cell segmentation pipeline.*


#### Instructions

- This notebook contains detailed instructions on how to program a pipeline that can segment cells in 2D fluorescence microscopy images of a developing tissue.


- Simply go through the instructions step by step and try to implement each step as best as you can.
    - By doing so, you will learn about key concepts of (bio-)image processing with python!


- If you are stuck...
    - ...first think some more about the problem and see if you can get yourself unstuck.
    - ...search the internet (in particular Stack Overflow) for a solution to your problem.
    - ...if you are working through this tutorial in class, ask one of the tutors for help.
    - ...if nothing else helps, you can have a look at the solutions (`image_analysis_tutorial_solutions.py`) for inspiration.
        - **Beware:** If you simply copy a solution, you will learn *nothing* in the process! Use the solutions only as an inspiration for implementing your own version and make sure you fully understand how the solution works!
        
        
- Some exercises are labeled as 'BONUS' exercises. It is up to you whether you'd like to complete them. 
    - If things are going slowly, you may want to skip them and come back to them later.


#### Background 

The aim is to construct a pipeline for the identification and segmentation of cells in 2D confocal fluorescence microscopy images of a tissue with labeled membranes. This is among the most common tasks in bio-image analysis and is often essential for the extraction of useful quantitative information from microscopy data.

The pipeline is constructed based on provided example images (see `example_data` directory), which are single-color spinning-disk confocal micrographs (objective: `40X 1.2NA W`) of cells in live zebrafish embryos in early development (~10h post fertilization), fluorescently labeled with a membrane-localized fusion protein (`mNeonGreen:Ggamma9`).

##  Setup <a id=setup></a>

#### Development Environment

- Depending on your preferences, there are several alternatives for implementing this pipeline:
    1. Implement it directly in this Jupyter notebook *[recommended]*.
    2. Implement it in your favorite Integrated Development Environment (IDE) like PyCharm.
    3. Old School: Implement it in a text editor (like Vim...) and run it on the terminal.


#### Example Data

Make sure that you have downloaded the example image data (`example_cells_1.tif` and `example_cells_2.tif`), which should be located in the directory `example_data`, which in turn should be in the same directory as this notebook.


#### Python version

- The solutions are provided in **python 3.6** (but any version of python 3 should work without changes)
- Everything shown in the solutions also works in python 2.7 with only very minor changes.
- To install python, jupyter notebook and the required modules, we recommend the **Anaconda Distribution**. The installer can be downloaded [here](https://www.anaconda.com/download/).


#### Required modules

- Make sure the following modules are installed before you get started:
    - numpy
    - scipy
    - matplotlib
    - scikit-image
- All required modules come pre-installed if you are using the **[Anaconda distribution](https://www.anaconda.com/download/)** of python.

*Everything ready? Yes? Then let's get started!*

----

## Importing Modules & Packages <a id=import></a>

Let's start by importing the package NumPy, which enables the manipulation of numerical arrays:

In [ ]:
import numpy as np

*<font color=red>Important note:</font> If you are not at all familiar with arrays and NumPy, we strongly recommend that you first complete an introductory tutorial on this topic before carrying on!*

Recall that, once imported, we can use functions/modules from the package, for example to create an array:

In [ ]:
a = np.array([1, 2, 3])

print(a)
print(type(a))

Note that the package is imported under a variable name (here `np`). You can freely choose this name yourself. For example, it would be just as valid (but not as convenient) to write:

```python
import numpy as lovelyArrayTool
a = lovelyArrayTool.array([1,2,3])
```

#### <font color='teal'>Exercise</font>

Using the import command as above, follow the instructions in the comments below to import two additional modules that we will be using frequently in this pipeline.

In [ ]:
# The plotting module matplotlib.pyplot as plt
### YOUR CODE HERE!
import matplotlib.pyplot as plt

# The image processing module scipy.ndimage as ndi
### YOUR CODE HERE!
import scipy.ndimage as ndi

#### Side Note for Jupyter Notebook Users

You can configure how the figures made by matplotlib are displayed.

The most common options are the following:

- **inline**:   displays as static figure in code cell output
- **notebook**: displays as interactive figure in code cell output 
- **qt**:       displays as interactive figure in a separate window

Feel free to test them out on one of the figures you will generate later on in the tutorial. The code cell below shows how to set the different options. Note that combinations of different options in the same notebook do not always work well, so it is best to decide for one and use it throughout. You may need to restart the kernel (`Kernel > Restart`) when you change from one option to another.

In [ ]:
# Set matplotlib backend
%matplotlib inline 
#%matplotlib notebook 
#%matplotlib qt 

## Loading & Handling Image Data <a id=load></a>

#### Background

Images are essentially just numbers (representing intensity) in an ordered grid of pixels. Image processing is simply to carry out mathematical operations on these numbers.

The ideal object for storing and manipulating ordered grids of numbers is the **array**. Many mathematical operations are well defined on arrays and can be computed quickly by vector-based computation.

Arrays can have any number of dimensions (or "axes"). For example, a 2D array could represent the x and y axis of a grayscale image (xy), a 3D array could contain a z-stack (zyx), a 4D array could also have multiple channels for each image (czyx) and a 5D array could have time on top of that (tczyx).

#### <font color='teal'>Exercise</font>

We will now proceed to load one of the example images and verify that we get what we expect. 

Note: Before starting, it always makes sense to have a quick look at the data in Fiji/ImageJ so you know what you are working with!

Follow the instructions in the comments below.

In [ ]:
# (i) Specify the directory path and file name

# Create a string variable with the name of the file you'd like to load (here: 'example_cells_1.tif').
# Suggested name for the variable: filename
# Note: Paths and filenames can contain slashes, empty spaces and other special symbols, which can cause 
#       trouble for programming languages under certain circumstances. To circumvent such trouble, add 
#       the letter r before your string definition to create a so-called 'raw string', which is not
#       affected by these problems (e.g. `my_raw_string = r"some string with funny symbols: \\\!/~***!"`).
### YOUR CODE HERE!
filename = r'example_cells_1.tif'

# If the file is not in the current working directory, you must also have a way of specifying the path
# to the directory where the file is stored. Most likely, your example images are stored in a directory
# called 'example_data' in the same folder as this notebook. Note that you can use either the full path
#  - something like r"/home/jack/data/python_image_analysis/example_data/example_cells_1.tif"
# or the relative path, starting from the current working directory
#   - here that would just be r"example_data"
# Create a string variable with the path to the directory that contains the file you'd like to load.
# Suggested name for the variable: dirpath
### YOUR CODE HERE!
dirpath = r'example_data'  # Relative path

In [ ]:
# (ii) Combine the directory path and file name into one variable, the file path

# Import the function 'join' from the module 'os.path'
# This function automatically takes care of the slashes that need to be added when combining two paths.
### YOUR CODE HERE!
from os.path import join

# Use the 'join' function to combine the directory path with the file name and create a new variable.
# Print the result to see that everything is correct (this is always a good idea!)
# Suggested name for the variable: filepath
### YOUR CODE HERE!
filepath = join(dirpath, filename)
print(filepath)

In [ ]:
# (iii) Load the image

# Import the function 'imread' from the module 'skimage.io'.
# (Note: If this gives you an error, please refer to the note below!)
### YOUR CODE HERE!
from skimage.io import imread

# Load 'example_cells_1.tif' and store it in a variable.
# Suggested name for the variable: img
### YOUR CODE HERE!
img = imread(filepath)

----

*Important note for those who get an error when trying to import `imread` from `skimage.io`:*

Some users have been experiencing problems with this module, even though the rest of skimage is installed correctly (running `import skimage` does not given an error). This may have something to do with operating system preferences. The easiest solution in this case is to install the module `tifffile` (with three`f`) and use the function `imread` from that module (it is identical to the `imread` function of `skimage.io` when reading `tif` files). 

The `tifffile` module does not come with the Anaconda distribution, so it's likely that you don't have it installed. To install it, save and exit Jupyter notebook, then go to a terminal and type `conda install -c conda-forge tifffile`. After the installation is complete, restart Jupyter notebook, come back here and import `imread` from `tifffile`. This should now hopefully work.

----

In [ ]:
# (iv) Check that everything is in order

# Check that 'img' is a variable of type 'ndarray' - use Python's built-in function 'type'.
### YOUR CODE HERE!
print("Loaded array is of type:", type(img))

# Print the shape of the array by looking at its 'shape' attribute. 
# Make sure you understand the output!
### YOUR CODE HERE!
print("Loaded array has shape:", img.shape)

# Check the datatype of the individual numbers in the array. You can use the array attribute 'dtype' to do so.
# Make sure you understand the output!
### YOUR CODE HERE!
print("Loaded values are of type:", img.dtype)

Note: The dtype should be 'uint8', because these are unsigned 8-bit integer images.

This means that the intensity values range from 0 to 255 in steps of 1.

In [ ]:
# (v) Look at the image to confirm that everything worked as intended

# To plot an array as an image, use pyplot's functions 'plt.imshow' followed by 'plt.show'. 
# Check the documentation for 'plt.imshow' and note the parameters that can be specified, such as colormap (cmap)
# and interpolation. Since you are working with scientific data, interpolation is unwelcome, so you should set it 
# to "none". The most common cmap for grayscale images is naturally "gray".
# You may also want to adjust the size of the figure. You can do this by preparing the figure canvas with
# the function 'plt.figure' before calling 'plt.imshow'. The canvas size is adjusted using the keyword argument
# 'figsize' when calling 'plt.figure'.
### YOUR CODE HERE!
plt.figure(figsize=(7,7))
plt.imshow(img, interpolation='none', cmap='gray')
plt.show()

## Preprocessing <a id=prepro></a>

#### Background

The goal of image preprocessing is to prepare or optimize the images to make further analysis easier. Usually, this boils down to increasing the signal-to-noise ratio by removing noise and background and by enhancing structures of interest.

The specific preprocessing steps used in a pipeline depend on the type of sample, the microscopy technique used, the image quality, and the desired downstream analysis. 

The most common operations include:


- Deconvolution
    - Image reconstruction based on information about the PSF of the microscope
    - These days deconvolution is often included with microscope software
    - *Our example images are not deconvolved, but will do just fine regardless*


- Conversion to 8-bit images to save memory / computational time
    - *Our example images are already 8-bit*


- Cropping of images to an interesting region
    - *The field of view in our example images is fine as it is*


- Smoothing of technical noise
    - This is a very common step and usually helps to improve almost any type of downstream analysis
    - Commonly used filters are the `Gaussian filter` and the `median filter`
    - *Here we will be using a Gaussian filter.*


- Corrections of technical artifacts
    - Common examples are uneven illumination and multi-channel bleed-through
    - *Here we will deal with uneven signal by adaptive/local thresholding*


- Background subtraction
    - There are various ways of sutracting background signal from an image
    - Two different types are commonly distinguished:
        - `uniform background subtraction` treats all regions of the image the same
        - `adaptive or local background subtraction` automatically accounts for differences between regions of the image
    - *Here we will do something similar to adaptive background subtraction when we do adaptive thresholding*

#### Gaussian Smoothing

A Gaussian filter smoothens an image by convolving it with a Gaussian-shaped kernel. In the case of a 2D image, the Gaussian kernel is also 2D and will look something like this:

<img src="ipynb_images\gaussian_kernel_grid.png" alt="Gaussian Kernel Figure" style="width: 300px;"/>

How much the image is smoothed by a Gaussian kernel is determined by the standard deviation  of the Gaussian distribution, usually referred to as **sigma** ($\sigma$). A higher $\sigma$ means a broader distribution and thus more smoothing.

**How to choose the correct value of $\sigma$?**

This depends a lot on your images, in particular on the pixel size. In general, the chosen $\sigma$ should be large enough to blur out noise but small enough so the "structures of interest" do not get blurred too much. Usually, the best value for $\sigma$ is simply found by trying out some different options and looking at the result. 

#### <font color='teal'>Exercise</font>

Perform Gaussian smoothing and visualize the result.

Follow the instructions in the comments below.

In [ ]:
# (i) Create a variable for the smoothing factor sigma, which should be an integer value

### YOUR CODE HERE!
sigma = 3

# After implementing the Gaussian smoothing function below, you can modify this variable 
# to find the ideal value of sigma.

In [ ]:
# (ii) Perform the smoothing on the image

# To do so, use the Gaussian filter function 'ndi.gaussian_filter' from the 
# image processing module 'scipy.ndimage', which was imported at the start of the tutorial. 
# Check out the documentation of scipy to see how to use this function. 
# Allocate the output to a new variable.
### YOUR CODE HERE!
img_smooth = ndi.gaussian_filter(img, sigma)

In [ ]:
# (iii) Visualize the result using 'plt.imshow'

# Compare with the original image visualized above. 
# Does the output make sense? Is this what you expected? 
# Can you optimize sigma such that the image looks smooth without blurring the membranes too much?
### YOUR CODE HERE!
plt.figure(figsize=(7,7))
plt.imshow(img_smooth, interpolation='none', cmap='gray')
plt.show()

# To have a closer look at a specific region of the image, crop that region out and show it in a 
# separate plot. Remember that you can crop arrays by "indexing" or "slicing" them similar to lists.
# Use such "zoomed-in" views throughout this tutorial to take a closer look at your intermediate 
# results when necessary.
### YOUR CODE HERE!
plt.figure(figsize=(6,6))
plt.imshow(img_smooth[400:600, 200:400], interpolation='none', cmap='gray')
plt.show()

In [ ]:
# (iv) BONUS: Show the raw and smoothed images side by side using 'plt.subplots'

### YOUR CODE HERE!
fig, ax = plt.subplots(1, 2, figsize=(10,7))
ax[0].imshow(img, interpolation='none', cmap='gray')
ax[1].imshow(img_smooth, interpolation='none', cmap='gray')
ax[0].set_title('Raw Image')
ax[1].set_title('Smoothed Image')
plt.show()